[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/marmalux/Clientes_TelecomX/blob/main/Clientes_TelecomX.ipynb)


# Análisis de evasión de clientes en Telecom X

## Importación de datos

In [186]:
import numpy as np
import pandas as pd
import requests
import json

#importar de una API y pasar a json
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
datos_api = requests.get(url)
datos_json=json.loads(datos_api.text)

In [107]:
# normalizar datos
datos_cliente = pd.json_normalize(datos_json)
datos_cliente.sample(5)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
1941,2748-MYRVK,Yes,Female,0,No,No,63,Yes,Yes,Fiber optic,...,Yes,No,No,Yes,Yes,One year,No,Bank transfer (automatic),99.90,6137
6595,9067-YGSCA,No,Female,0,No,No,70,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),26.05,1856.4
6138,8400-WZICQ,No,Female,0,Yes,Yes,55,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,No,Electronic check,55.70,3131.8
5140,7017-VFHAY,No,Female,0,Yes,Yes,61,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),115.10,7008.15
6865,9477-LGWQI,No,Male,0,Yes,Yes,70,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),80.15,5600.15


### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

Comprobación de datos

In [108]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [180]:
# print(datos_cliente['Churn'].unique()) # en la columna Churn hay valores incoherentes

# datos_cliente['Churn'].str.strip()
(datos_cliente[(datos_cliente['Churn'] == '')].head())
# (datos_cliente[(datos_cliente['account.Charges.Total'] == ' ')].head())


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45


Después de revisar, la columna *Churn* y la columna *account.Charges.Total* tienen valores inconsistentes, *Churn* faltan  valores de 'Yes' pues depués de analizar, hay gastos totales por lo que debe ser aun cliente de la empresa, y la columna de gastos totales *account.Charges.Total* tiene valores vacios no nulos, pero como son clientes que ya no estan en la empresa se puden dejar como 0 para poder cambiar el tipo de dato

In [183]:
# transformar columna Churn
datos_cliente['Churn'] = datos_cliente['Churn'].replace('',pd.NA).fillna('No')# se remplazan las celdas vacias a valores nulos y despues se rellenan esos valores con 'No'

#transformar columna 	account.Charges.Total
datos_cliente['account.Charges.Total'] = datos_cliente['account.Charges.Total'].replace(' ',pd.NA).fillna(0)

In [184]:
#Pasar a tipo float los datos de los cargos totales
datos_cliente['account.Charges.Total']=datos_cliente['account.Charges.Total'].astype(float64)

In [185]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

### Formatear nombres de columnas para mejor entendimiento en presentación

In [189]:

columnas=['customerID',
          'Still_Customer',
          'Gender',
          'Senior_Citizen',
          'Partner',
          'Dependents',
          'Contract_Months',
          'Phone_Service',
          'Multiple_Lines',
          'Internet_Service',
          'Online_Security',
          'Online_Backup',
          'Device_Protection',
          'Tech_Support',
          'Streaming_TV',
          'Streaming_Movies',
          'Contract_type',
          'Paperless_Billing',
          'Payment_Method',
          'Charges_Monthly',
          'Charges_Total']
datos_cliente.columns = columnas

Crear columa de cargos al día "Charges_Daily"

In [192]:
#
gastos_diarios=datos_cliente['Charges_Monthly'].to_numpy()
gastos_diarios = gastos_diarios/30
datos_cliente['Charges_Day'] = np.round(gastos_diarios,2)

In [193]:
datos_cliente.head()

,customerID,Still_Customer,Gender,Senior_Citizen,Partner,Dependents,Contract_Months,Phone_Service,Multiple_Lines,Internet_Service,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract_type,Paperless_Billing,Payment_Method,Charges_Monthly,Charges_Total,Charges_Day
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.80
